# Read the records from the Simple Test of Reciprocity

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from sys import argv
from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize

from scipy import signal


from pyaspect.project import *
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import *
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import *
from pyaspect.specfemio.write import *
from pyaspect.specfemio.write import _write_header
from pyaspect.specfemio.read import _read_headers
from pyaspect.specfemio.read import *
from pyaspect.specfemio.utils import *


import pyaspect.events.gevents as gevents
import pyaspect.events.gstations as gstations
from pyaspect.events.munge.knmi import correct_station_depths as csd_f
import pyaspect.events.mtensors as mtensors
from obspy.imaging.beachball import beach
from obspy import UTCDateTime
import shapefile as sf

## Set Project Paths

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
projects_fqp = os.path.join(data_out_dir,'tmp','TestProjects','Computed_Forward_and_Reciprocity_Test')
recip_project_fqp = os.path.join(projects_fqp,'ReciprocalTestProject')
fwd_project_fqp = os.path.join(projects_fqp,'ForwardTestProject')
!ls {recip_project_fqp}
print()
!ls {fwd_project_fqp}

## Define bandpass (this is just a secondary test to sos filter) (taken from scipy recipies)

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y



## Read Reciprocal Project RecordHeader and load data

In [ ]:
import glob
recip_record_fqp = os.path.join(recip_project_fqp,'pyheader.project_record')
recip_record_h = _read_headers(recip_record_fqp)

ne = recip_record_h.nevents
ns = recip_record_h.nsrc

'''
''';
l_data_x = []
l_data_y = []
l_data_z = []
for idx, row in recip_record_h.stations_df.iterrows():
    #fp_prefix = row['data_fqdn'].values[0]
    fp_prefix = row['data_fqdn']
    fp = os.path.join(projects_fqp,fp_prefix) 
    match_fp = fp + '.*X[XYZEN].sem*'
    for filepath in glob.glob(match_fp):
        #print(f'filepath.split(): {filepath.split(".")}')
        comp = filepath.split('.')[-2][-1]
        #print(f'comp: {comp}')
        #print(f'np.fromfile({filepath}, dtype=np.float32)')
        if comp == 'X':
            #l_data_x.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
            l_data_x.append(np.fromfile(filepath, dtype=np.float32))
        elif comp == 'Y':
            #l_data_y.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
            l_data_y.append(np.fromfile(filepath, dtype=np.float32))
        elif comp == 'Z':
            #l_data_z.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
            #FIXME: -1* ? ...the Zup force
            l_data_z.append(-1*np.fromfile(filepath, dtype=np.float32))
            #l_data_z.append(np.fromfile(filepath, dtype=np.float32))
        else:
            assert False
            
recip_record_h.stations_df['comp_EX'] = l_data_x
recip_record_h.stations_df['comp_NY'] = l_data_y
recip_record_h.stations_df['comp_Z']  = l_data_z

'''
#print(f'Record w/ data:\n\n{recip_record_h.stations_df}')
for idx, row in recip_record_h.stations_df.iterrows():
    fp_prefix = row['data_fqdn']
    x_pre = row['comp_EX'][0]
    y_pre = row['comp_NY'][0]
    z_pre = row['comp_Z'][0]
    assert x_pre == fp_prefix
    assert y_pre == fp_prefix
    assert z_pre == fp_prefix
    #print(f'row["comp_EX"] = {row["comp_EX"]}')
''';
    
print(f'Record w/ data:\n\n{recip_record_h.stations_df["comp_Z"]}')

num_tr = len(recip_record_h.stations_df)//7
print(f'num_tr: {num_tr}')

fig, axs = plt.subplots(num_tr,1,figsize=(15,2*num_tr))
#fig, axs = plt.subplots(4,1,figsize=(15,3*4))

ip = 0
it = 0
for idx, row in recip_record_h.stations_df.iterrows():
    #sos = signal.butter(4, [1,5], 'bp', fs=1000, output='sos')
    #filtered = signal.sosfilt(sos, row['comp_Z'])
    filtered = butter_bandpass_filter(row['comp_Z'], 1, 10, 1000, order=4)
    axs[ip].plot(filtered,linewidth=3)
    #axs[ip].plot(row['comp_EX'],title='EX')
    #axs[ip].plot(row['comp_NY'],title='NY')
    #axs[ip].plot(row['comp_Z'],title='Z')
    it += 1
    ip = it//7
plt.show()

    

'''
for eidx, edf in recip_record_h.stations_df.groupby(level='eid'):
    assert ns == edf.index.get_level_values('sid').nunique()
    for sidx, sdf in edf.groupby(level='sid'):
        for ridx, rdf in sdf.groupby(level='trid'):
            for gidx, gdf in rdf.groupby(level='gid'):
                fp_prefix = gdf['data_fqdn'].values[0]
                fp = os.path.join(projects_fqp,fp_prefix) 
                for comp in ['X','Y','Z']:
                    match_fp = fp + '.*X' + comp + '.semd'
                    cname = 'comp_' + comp
                    for filepath in glob.glob(match_fp):
                        #print(f'np.fromfile({filepath}, dtype=np.float32)')
                        gdf[cname] = np.fromfile(filepath, dtype=np.float32)
        

print(recip_record_h)
''';
    
    
'''
l_traces = []
for fp in recip_record_h.stations_df['data_fqdn'].tolist():
    
    fp_prefix = os.path.join(projects_fqp,fp) 
    match_fp = fp_prefix + '.*XZ.semd'
    #print(f'fp_prefix: {fp_prefix}')
    #print(f'match_fp: {match_fp}')
    
    for filepath in glob.glob(match_fp):
        #print(f'files:\n{filepath}')
        zar = np.fromfile(filepath, dtype=np.float32)
        #print(f'zar.shape = {zar.shape}')
        l_traces.append(zar)

traces_z = np.array(l_traces)
print(f'traces_z.shape: {traces_z.shape}')
''';
    
      
'''
for idx, rec in recip_record_h.stations_df['data_fqdn'].iterrows():
    print(f'idx: {idx}')
    print(f'rec:\n{rec}')
''';

'''
plt.plot(traces_z[100])
plt.show()
''';

## function for computing the derivatives and applying the bandpass to reciprocal traces

In [ ]:
def calculate_mt_trace(eidx,sidx,tidx,g_p1,g_m1,df,sos,comp_key,coord_key):
    gidx_0  = pd.IndexSlice[eidx,sidx,tidx,0]
    gidx_p1 = pd.IndexSlice[eidx,sidx,tidx,g_p1]
    gidx_m1 = pd.IndexSlice[eidx,sidx,tidx,g_m1]
    df_0    = df.loc[gidx_0]
    df_p1   = df.loc[gidx_p1]
    df_m1   = df.loc[gidx_m1]
    data_p1 = signal.sosfilt(sos, df_p1[comp_key])
    data_m1 = signal.sosfilt(sos, df_m1[comp_key])
    #data_p1 = butter_bandpass_filter(df_p1[comp_key], 0.25, 20, 1000, order=4)
    #data_m1 = butter_bandpass_filter(df_m1[comp_key], 0.25, 20, 1000, order=4)
    c_p1    = df_p1[coord_key]
    c_m1    = df_m1[coord_key]
    c_0     = df_0[coord_key]
    h       = c_p1 - c_m1
    delta   = 0.5*h
    c       = c_m1 + delta
    
    assert h != 0
    assert c_0-c == 0
    
    h_scale  = 1/h
    mt_trace = h_scale*(data_p1 - data_m1)
    
    #return gidx_0, mt_trace
    #swap trid and eidx 
    return (tidx,eidx,comp_key), mt_trace


## sort the traces into xx,yy,zz,xy,xz,yz for each event (note: sid=0,1,2 => x,y,z)

## Similar to cell directly above, but calculate full 9D Greens Functions

In [ ]:
comp_dict = {'comp_EX':0,'comp_NY':1,'comp_Z':2}
coord_dict = {0:'lon_xc',1:'lat_yc',2:'depth'}
sos = signal.butter(3, [1,5], 'bp', fs=1000, output='sos')

ns = 3    #mum event sources
ng = 4    #num sations
nc = 3    #num components (x=0,y=1,z=0)
nf = 3    #num force-directions (ex=0,ny=1,zup=2)
nd = 3    #num direction/derivatives (d_ex=0,d_ny=1,d_zup=2)
nt = 4096 #num samples in trace/greens function (hard coded for testing only)
rgf_table = np.zeros((ne,ng,nc,nf,nd,nt))

for eidx, edf in recip_record_h.stations_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        for tidx, tdf in sdf.groupby(level='trid'):
            for comp_key in comp_dict.keys():
                ie = tidx
                ig = eidx
                ic = comp_dict[comp_key]
                fi = sidx
                
                # dx=0,dy=1,dz=2
                for di in range(3):
                    coord_key = coord_dict[di]
                    ip1 = di+1     #coord + h
                    im1 = ip1 + 3  #coord - h
                    #print(f'ie:{ie},ig:{ig},ic:{ic},fi:{fi},di:{di},ip1:{ip1},im1:{im1}')
                    #rgf_table[0,0,0,2,1]
                    #rgf_table[0,0,0,1,2]
                    key, rgf_table[ie,ig,ic,fi,di,:] = calculate_mt_trace(eidx,sidx,tidx,ip1,im1,tdf,sos,comp_key,coord_key)
                
assert False               

## Plot and compare table

In [ ]:
%matplotlib inline
#%matplotlib notebook
clrs = ['red','green','blue']

ie = 2
ig = 3

fig, axs = plt.subplots(18,1,figsize=(11,18*3))
fig.subplots_adjust(hspace=.35)

################################################
#
# M1
#
################################################

axs[0].plot(rgf_table[ie,ig,0,0,0],color=clrs[0],label='G:x,xx')
axs[0].legend()
axs[0].set_title('M1 -- E:x,xx')

axs[1].plot(rgf_table[ie,ig,0,1,0],color=clrs[0],label='G:x,yx')
axs[1].legend()
axs[1].set_title('M1 -- E:y,xx')

axs[2].plot(rgf_table[ie,ig,0,2,0],color=clrs[0],label='G:x,zx')
axs[2].legend()
axs[2].set_title('M1 -- E:z,xx')

################################################
#
# M2
#
################################################

axs[3].plot(rgf_table[ie,ig,1,0,1],color=clrs[0],label='G:y,xy')
axs[3].legend()
axs[3].set_title('M2 -- E:x,yy')

axs[4].plot(rgf_table[ie,ig,1,1,1],color=clrs[0],label='G:y,yy')
axs[4].legend()
axs[4].set_title('M2 -- E:y,yy')

axs[5].plot(rgf_table[ie,ig,1,2,1],color=clrs[0],label='G:y,zy')
axs[5].legend()
axs[5].set_title('M2 -- E:z,yy')

################################################
#
# M3
#
################################################

axs[6].plot(rgf_table[ie,ig,2,0,2],color=clrs[0],label='G:z,xz')
axs[6].legend()
axs[6].set_title('M3 -- E:x,zz')

axs[7].plot(rgf_table[ie,ig,2,1,2],color=clrs[0],label='G:z,yz')
axs[7].legend()
axs[7].set_title('M3 -- E:y,zz')

axs[8].plot(rgf_table[ie,ig,2,2,2],color=clrs[0],label='G:z,zz')
axs[8].legend()
axs[8].set_title('M3 -- E:z,zz')

################################################
#
# M4/M7
#
################################################
axs[9].plot(rgf_table[ie,ig,0,0,1],color=clrs[0],label='G:x,xy')
axs[9].plot(rgf_table[ie,ig,1,0,0],color=clrs[1],label='G:y,xx')
axs[9].legend()
axs[9].set_title('M4/M7 -- E:x,xy')

axs[10].plot(rgf_table[ie,ig,0,1,1],color=clrs[0],label='G:x,yy')
axs[10].plot(rgf_table[ie,ig,1,1,0],color=clrs[1],label='G:y,xy')
axs[10].legend()
axs[10].set_title('M4/M7 -- E:y,xy')

axs[11].plot(rgf_table[ie,ig,0,2,1],color=clrs[0],label='G:x,zy')
axs[11].plot(rgf_table[ie,ig,1,2,0],color=clrs[1],label='G:y,zx')
axs[11].legend()
axs[11].set_title('M4/M7 -- E:z,xy')

################################################
#
# M5/M8
#
################################################

axs[12].plot(rgf_table[ie,ig,0,0,2],color=clrs[0],label='G:x,xz')
axs[12].plot(rgf_table[ie,ig,2,0,0],color=clrs[1],label='G:z,xx')
#axs[12].plot(rgf_table[ie,ig,0,2,0],color=clrs[1],label='G:z,xx')
axs[12].legend()
axs[12].set_title('M5/M8 -- E:x,xz')

axs[13].plot(rgf_table[ie,ig,0,1,2],color=clrs[0],label='G:x,yy')
axs[13].plot(rgf_table[ie,ig,2,1,0],color=clrs[1],label='G:z,yx')
#axs[13].plot(rgf_table[ie,ig,0,2,1],color=clrs[1],label='G:z,yx')
axs[13].legend()
axs[13].set_title('M5/M8 -- E:y,xz')

axs[14].plot(rgf_table[ie,ig,0,2,2],color=clrs[0],label='G:x,zz')
axs[14].plot(rgf_table[ie,ig,2,2,0],color=clrs[1],label='G:z,zx')
axs[14].legend()
axs[14].set_title('M5/M8 -- E:z,xz')

################################################
#
# M6/M9
#
################################################

axs[15].plot(rgf_table[ie,ig,1,0,2],color=clrs[0],label='G:y,xz')
axs[15].plot(rgf_table[ie,ig,2,0,1],color=clrs[1],label='G:z,xy')
axs[15].legend()
axs[15].set_title('M6/M9 -- E:x,yz')

axs[16].plot(rgf_table[ie,ig,1,1,2],color=clrs[0],label='G:y,yz')
axs[16].plot(rgf_table[ie,ig,2,1,1],color=clrs[1],label='G:z,yy')
axs[16].legend()
axs[16].set_title('M6/M9 -- E:y,yz')

axs[17].plot(rgf_table[ie,ig,1,2,2],color=clrs[0],label='G:y,zz')
axs[17].plot(rgf_table[ie,ig,2,2,1],color=clrs[1],label='G:z,zy')
axs[17].legend()
axs[17].set_title('M6/M9 -- E:z,yz')

plt.show()


## Read Forward Record and load data

In [ ]:
fwd_record_fqp = os.path.join(fwd_project_fqp,'pyheader.project_record')
fwd_record_h = _read_headers(fwd_record_fqp)

ne = fwd_record_h.nevents
ns = fwd_record_h.nsrc

'''
''';
l_data_x = []
l_data_y = []
l_data_z = []
for idx, row in fwd_record_h.stations_df.iterrows():
    #fp_prefix = row['data_fqdn'].values[0]
    fp_prefix = row['data_fqdn']
    fp = os.path.join(projects_fqp,fp_prefix) 
    match_fp = fp + '.*X[XYZEN].sem*'
    for filepath in glob.glob(match_fp):
        #print(f'filepath.split(): {filepath.split(".")}')
        comp = filepath.split('.')[-2][-1]
        #print(f'comp: {comp}')
        #print(f'np.fromfile({filepath}, dtype=np.float32)')
        if comp == 'X':
            #l_data_x.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
            l_data_x.append(np.fromfile(filepath, dtype=np.float32))
        elif comp == 'Y':
            #l_data_y.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
            l_data_y.append(np.fromfile(filepath, dtype=np.float32))
        elif comp == 'Z':
            #l_data_z.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
            l_data_z.append(np.fromfile(filepath, dtype=np.float32))
        else:
            assert False
            
fwd_record_h.stations_df['comp_EX'] = l_data_x
fwd_record_h.stations_df['comp_NY'] = l_data_y
fwd_record_h.stations_df['comp_Z']  = l_data_z

print(f'Forward Record:\n{fwd_record_h}')

## Inspec moment tensors from Forward event. Will use thise for making Reciprocal Traces

In [ ]:
#print(f'Forward Record Sources:\n{fwd_record_h.solutions_df}')
SrcHeader = fwd_record_h.solution_cls

d_fwd_src = {}
for eidx, edf in fwd_record_h.solutions_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        idx = pd.IndexSlice[eidx,sidx]
        src = SrcHeader.from_series(fwd_record_h.solutions_df.loc[idx])
        mag    = src.mw
        strike = src.strike
        dip    = src.dip
        rake   = src.rake
        mt = MomentTensor(mw=mag,strike=strike,dip=dip,rake=rake)
        d_fwd_src[eidx] = mt
        #print(f'mt.aki_m6:\n{mt.aki_richards_m6()}')
        #print(f'header.m6:\n{src.mt}\n')

for key in d_fwd_src:
    print(d_fwd_src[key])

## Derive the x,y, and z components for each event from the sorted reciprocity traces

In [ ]:
'''
Mtt = Mxx 
Mpp = Myy 
Mrr = Mzz 
Mtp = Mxy * -1.0  # sign change
Mrt = Mxz 
Mrp = Myz * -1.0  # sign change
''';

#make unit traces
'''
d_vcmt_xx
d_vcmt_yy
d_vcmt_zz
d_vcmt_xy
d_vcmt_xz
d_vcmt_yz
'''

cmb_traces = np.zeros((3,4,3,4096))
for ie in range(3):
    mt_arr = d_fwd_src[ie].aki_richards_unit_m6()
    wxx = mt_arr[0]
    wyy = mt_arr[1]
    wzz = mt_arr[2]
    wxy = mt_arr[3]
    wxz = mt_arr[4]
    wyz = mt_arr[5]
    for it in range(4):
        icomp = 0
        for comp_key in ['comp_EX','comp_NY','comp_Z']:
            
            cmb_traces[ie,it,icomp,:]  =   wxx*rgf_table[ie,it,0,icomp,0,:] #Mxx
            cmb_traces[ie,it,icomp,:] +=   wyy*rgf_table[ie,it,1,icomp,1,:] #Myy
            cmb_traces[ie,it,icomp,:] +=   wzz*rgf_table[ie,it,2,icomp,2,:] #Myy
            cmb_traces[ie,it,icomp,:] += wxy*rgf_table[ie,it,0,icomp,1,:] #Mxy
            cmb_traces[ie,it,icomp,:] += wxz*rgf_table[ie,it,0,icomp,2,:] #Mxz
            cmb_traces[ie,it,icomp,:] += wyz*rgf_table[ie,it,1,icomp,2,:] #Myz
            '''
            cmb_traces[ie,it,icomp,:]  = wxx*d_vcmt_xx[(ie,it,comp_key)] #Mxx
            cmb_traces[ie,it,icomp,:] += wyy*d_vcmt_yy[(ie,it,comp_key)] #Myy
            cmb_traces[ie,it,icomp,:] += wzz*d_vcmt_zz[(ie,it,comp_key)] #Mzz
            cmb_traces[ie,it,icomp,:] += wxy*d_vcmt_xy[(ie,it,comp_key)] #Mxy
            cmb_traces[ie,it,icomp,:] += wxz*d_vcmt_xz[(ie,it,comp_key)] #Mxz
            cmb_traces[ie,it,icomp,:] += wyz*d_vcmt_yz[(ie,it,comp_key)] #Myz
            ''';
            '''
            cmb_traces[ie,it,icomp,:]  = wyy*d_vcmt_xx[(ie,it,comp_key)] #Mxx
            cmb_traces[ie,it,icomp,:] += wxx*d_vcmt_yy[(ie,it,comp_key)] #Myy
            cmb_traces[ie,it,icomp,:] += wzz*d_vcmt_zz[(ie,it,comp_key)] #Mzz
            cmb_traces[ie,it,icomp,:] += 2*wxy*d_vcmt_xy[(ie,it,comp_key)] #Mxy
            cmb_traces[ie,it,icomp,:] += 2*wyz*d_vcmt_xz[(ie,it,comp_key)] #Mxz
            cmb_traces[ie,it,icomp,:] += 2*wxz*d_vcmt_yz[(ie,it,comp_key)] #Myz
            ''';
            icomp += 1
        

## Plot the x,y,z compents for each trace of each event

In [ ]:
ne  = 3
ntr = 4
nc  = 3
nplt = ne*ntr*nc 
fig, axs = plt.subplots(nplt,1,figsize=(15,2*nplt))

ip = 0
for ie in range(ne):
    for itr in range(ntr):
        for ic in range(nc):
            axs[ip].plot(cmb_traces[ie,itr,ic,:],linewidth=5)
            ip += 1

assert ip == nplt
plt.show()

## bandpass the "forward" traces to match the bandpass of the reciprocal traces

In [ ]:
fwd_traces = np.zeros((3,4,3,4096))

#conv_s = (10**(-11))
conv_s = 1
sos = signal.butter(4, [1,5], 'bp', fs=1000, output='sos')
for eidx, edf in fwd_record_h.stations_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        for tidx, tdf in sdf.groupby(level='trid'):
            idx = pd.IndexSlice[eidx,sidx,tidx,0]
            ip = 0
            for comp_key in ['comp_EX','comp_NY','comp_Z']:
                #print(tdf.loc[idx,comp_key])
                #fwd_traces[eidx,tidx,ip,:] =  conv_s*signal.sosfilt(sos, tdf.loc[idx,comp_key])
                fwd_traces[eidx,tidx,ip,:] =  conv_s*butter_bandpass_filter(tdf.loc[idx,comp_key], 1, 3, 1000, order=3)
                #fwd_traces[eidx,tidx,ip,:] =  conv_s*tdf.loc[idx,comp_key]
                ip += 1

## Plot the "forward" traces (black) on top of the reciprocal constructed traces (fat-blue)

In [ ]:
ne  = 3
ntr = 4
nc  = 3
nplt = ne*ntr*nc 
fig, axs = plt.subplots(nplt,1,figsize=(15,2*nplt))

ip = 0
for ie in range(ne):
    for itr in range(ntr):
        for ic in range(nc):
            norm_traces =  cmb_traces[ie,itr,ic,:]/np.max(np.abs(cmb_traces[ie,itr,ic,:]))
            #axs[ip].plot(cmb_traces[ie,itr,ic,:],linewidth=5,zorder=0)
            axs[ip].plot(norm_traces,linewidth=5,zorder=0)
            grad_traces = np.gradient(fwd_traces[ie,itr,ic,:])
            grad_traces /= np.max(np.abs(grad_traces))
            #axs[ip].plot(np.gradient(fwd_traces[ie,itr,ic,:]),color='black',linewidth=1,zorder=1)
            axs[ip].plot(grad_traces,color='yellow',linewidth=2,zorder=1)
            ip += 1

assert ip == nplt
plt.show()

# Everthing below this point is testing bandpass filtering

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def run():
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.signal import freqz

    # Sample rate and desired cutoff frequencies (in Hz).
    #fs = 5000.0
    #lowcut = 500.0
    #highcut = 1250.0
    fs = 1000
    lowcut = 1.0
    highcut = 10.0

    # Plot the frequency response for a few different orders.
    plt.figure(1)
    plt.clf()
    for order in [3, 6, 9]:
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        w, h = freqz(b, a, worN=2000)
        plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

    plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
             '--', label='sqrt(0.5)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain')
    plt.grid(True)
    plt.legend(loc='best')

    # Filter a noisy signal.
    T = 0.05
    nsamples = int(T * fs)
    t = np.linspace(0, T, nsamples, endpoint=False)
    a = 0.02
    f0 = 600.0
    x = 0.1 * np.sin(2 * np.pi * 1.2 * np.sqrt(t))
    x += 0.01 * np.cos(2 * np.pi * 312 * t + 0.1)
    x += a * np.cos(2 * np.pi * f0 * t + .11)
    x += 0.03 * np.cos(2 * np.pi * 2000 * t)
    plt.figure(2)
    plt.clf()
    plt.plot(t, x, label='Noisy signal')

    y = butter_bandpass_filter(x, lowcut, highcut, fs, order=6)
    plt.plot(t, y, label='Filtered signal (%g Hz)' % f0)
    plt.xlabel('time (seconds)')
    plt.hlines([-a, a], 0, T, linestyles='--')
    plt.grid(True)
    plt.axis('tight')
    plt.legend(loc='upper left')

    plt.show()


run()

In [ ]:
# Filter a noisy signal.
t = np.linspace(0, 1, 1001, endpoint=True)
print(len(t))
print(t[1] -t[0])
print(t[-1])
hz0 = 5
hz1 = 20
x   = np.zeros_like(t)
xx  = np.zeros_like(t)
x  += 0.5*np.sin(hz0*2*np.pi*t)
xx += 0.5*np.sin(hz1*2*np.pi*t)
xxx = x + xx
plt.figure(1,figsize=(12,5))
plt.clf()
plt.plot(t, xxx, label='Cos')
#plt.plot(t, x)
#plt.plot(t, xx)



In [ ]:
y = butter_bandpass_filter(xxx, 15.0, 25, 1000, order=4)
yy = butter_bandpass_filter(xxx, 10.0, 30, 1000, order=4)
yyy = butter_bandpass_filter(xxx, 15.0, 25, 1000, order=4)

plt.figure(1,figsize=(12,5))
plt.clf()
plt.plot(t, xx, label='Cos')
plt.plot(t, y, label='Cos')
#plt.plot(t, yy, label='Cos')
#plt.plot(t, yyy, label='Cos')

In [ ]:
2**12